In [1]:
# Imports
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go

from datetime import datetime

from pandas_datareader import data as web

In [2]:
def dividendos_acao(acao: str):
    # Definindo características da página
    url = f'https://fundamentus.com.br/proventos.php?papel={acao}&tipo=2'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

    # Fazendo a request
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read()

    # Tratando alguns possíveis erros
    except HTTPError as e:
        print('HTTPError\n\n')

    except URLError as e:
        print('URLError\n\n')

    # Instanciando um objeto BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Pegando os nomes das colunas da tabela
    colunas_names = [col.getText() for col in soup.find('table', {'id': 'resultado'}).find('thead').findAll('th')]
    colunas = {i: col.getText() for i, col in enumerate(soup.find('table', {'id': 'resultado'}).find('thead').findAll('th'))}

    # Criando um DataFrame com os nomes das colunas
    dados = pd.DataFrame(columns=colunas_names)

    # Pegando os dados da tabela por linha
    for i in range(len(soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr'))):
        linha = soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr')[i].getText().split('\n')[1:]
        inserir_linha = pd.DataFrame(linha).T.rename(columns=colunas)
        dados = pd.concat([dados, inserir_linha], ignore_index=True)
        dados['date'] = dados.Data.map(lambda x: x.split()[0])
        dados['date'] = pd.to_datetime(dados.date)
        dados['valor'] = dados.Data.map(lambda x: x.split()[1])
        dados['valor'] = dados.valor.map(lambda x: float(x.replace(',', '.')))

    # Rearranjando o df
    dados = dados.drop(columns=['Data'])
    dados['tipo'] = dados.Valor
    dados = dados.drop(columns=['Valor'])
    dados['data_pagamento'] = dados.Tipo
    dados = dados.drop(columns=['Tipo', 'Data de Pagamento', 'Por quantas ações'])

    # É tetra!
    return dados

In [15]:
dividendos_acao('PETR4')

,date,valor,tipo,data_pagamento
0,2021-04-14,0.7874,DIVIDENDO,29/04/2021
1,2020-07-22,0.0004,DIVIDENDO,15/12/2020
2,2019-12-26,0.4200,JRS CAP PROPRIO,07/02/2020
3,2019-11-11,0.2000,JRS CAP PROPRIO,07/02/2020
4,2019-12-08,0.2000,JRS CAP PROPRIO,04/10/2019
...,...,...,...,...
65,1998-03-24,0.6694,DIVIDENDO,-
66,1998-03-24,0.2049,DIVIDENDO,-
67,1997-03-21,2.1263,DIVIDENDO,-
68,1997-03-21,2.0000,DIVIDENDO,-


In [3]:
def stonk_last_value(acao: str, start_date: str = None):
    if start_date == None:
        start_date = '01-01-2000'    
    # importar dados para o DataFrame
    return web.DataReader(f'{acao}.SA', data_source='yahoo', start=start_date)

In [4]:
def dividendos_fii(fii: str):
    # Definindo características da página
    url = f'https://fundamentus.com.br/fii_proventos.php?papel={fii}&tipo=2'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

    # Fazendo a request
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read()

    # Tratando alguns possíveis erros
    except HTTPError as e:
        print('HTTPError\n\n')

    except URLError as e:
        print('URLError\n\n')

    # Instanciando um objeto BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Pegando os nomes das colunas da tabela
    colunas_names = [col.getText() for col in soup.find('table', {'id': 'resultado'}).find('thead').findAll('th')]
    colunas = {i: col.getText() for i, col in enumerate(soup.find('table', {'id': 'resultado'}).find('thead').findAll('th'))}

    # Criando um DataFrame com os nomes das colunas
    dados = pd.DataFrame(columns=colunas_names)

    # Pegando os dados da tabela por linha
    for i in range(len(soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr'))):
        linha = soup.find('table', {'id': 'resultado'}).find('tbody').findAll('tr')[i].getText().split('\n')[1:-1]
        inserir_linha = pd.DataFrame(linha).T.rename(columns=colunas)
        dados = pd.concat([dados, inserir_linha], ignore_index=True)
        dados['date'] = pd.to_datetime(dados['Última Data Com'])
        dados['valor'] = dados.Valor.map(lambda x: float(x.replace(',', '.')))        

    dados['tipo'] = dados['Tipo']
    dados['data_pagamento'] = dados['Data de Pagamento']
    dados = dados.drop(columns=['Tipo', 'Última Data Com', 'Valor', 'Data de Pagamento'])

    # É tetra!
    return dados

In [5]:
def plot_by_dividendo(codigo_ativo: str, kind: str):
    if kind == 'fii':
        df = dividendos_fii(codigo_ativo)
    elif kind == 'acao':
        df = dividendos_acao(codigo_ativo)
    else:
        raise ValueError (f'Os valores aceitáveis são: fii e acao')
    # Gerando grafico para a acao
    fig, ax = plt.subplots(figsize=(25,8))
    conv_date = df.date.sort_values().map(lambda x: x.strftime('%d-%m-%Y'))
    fig = sns.barplot(x=conv_date, y=df.valor, data=df)
    title = f'{codigo_ativo} - dividendo/mes'
    fig.set_title(title.upper(), fontsize=25)
    plt.grid()
    plt.xticks(rotation=90)
    fig = fig

In [6]:
def plot_by_year(codigo_ativo: str, kind: str):
    if kind == 'fii':
        df = dividendos_fii(codigo_ativo)
    elif kind == 'acao':
        df = dividendos_acao(codigo_ativo)
    else:
        raise ValueError (f'Os valores aceitáveis são: fii e acao')
    # Gerando grafico para a acao
    fig, ax = plt.subplots(figsize=(25,8))
    fig = sns.barplot(x=df.date.map(lambda x: x.year), y=df.valor, data=df)
    title = f'{codigo_ativo} - dividendo/ano'
    fig.set_title(title.upper(), fontsize=25)
    plt.grid()
    plt.xticks(rotation=90)
    fig = fig

# ACOES

In [7]:
def my_acao(plot = None):
    # Acoes da minha carteira
    acoes: list = [
        'taee4',
        'enbr3',
        'csmg3',
        'itsa4',
        'enat3',
        'cmig4',
        'bbse3',
        'trpl4',
        'vale3',
        'wege3',
    ]
    # Criando listas para armazenar os valores
    median_list: list = []
    last_value_list: list = []
    magic_number_list: list = []
    price_magic_number_list: list = []

    for acao in acoes: 
        # Ultima valor de fechamento da acao
        stonk_value: float = round(stonk_last_value(acao, '01-01-2015').iloc[-1].Close, 2)
        last_value_list.append(stonk_value)

        # Media dos dividendos por mes
        df = dividendos_acao(acao).query('date > 2015')
        median_by_year = df.groupby(df.date.map(lambda x: x.year)).median()
        median_by_month_value = float(median_by_year.median()[0] / 12)
        median_list.append(median_by_month_value)

        # magic_number: valor_acao / dividendo_mes
        magic_number_value: int = int(stonk_value / median_by_month_value)
        magic_number_list.append(magic_number_value)

        # Valor para o MG
        price_magic_number: float = round(stonk_value * magic_number_value, 2)
        price_magic_number_list.append(price_magic_number)

        # Plots
        if plot:
            plot_by_dividendo(acao, 'acao')
            plot_by_year(acao, 'acao')

    # Criando o df
    df_acoes = pd.DataFrame(acoes, columns=['code'])
    df_acoes['last_value'] = pd.DataFrame(last_value_list)
    df_acoes['median'] = pd.to_numeric(median_list)
    df_acoes['magic_number'] = pd.to_numeric(magic_number_list)
    df_acoes['price_magic_number'] = pd.to_numeric(price_magic_number_list)

    return df_acoes

# FUNDOS IMOBILIARIOS

In [8]:
def my_fii(plot = None):
    # FIIs da minha carteira
    fiis: list = [
        'alzr11',
        'bcff11',
        'hglg11',
        'xplg11',
        'xpin11',
        'mxrf11',
        'hfof11',
        'rect11',
        'hctr11',
        'knip11',
        'irdm11',
        'vino11',
    ]
    # Criando listas para armazenar os valores
    median_fii_list: list = []
    last_fii_value_list: list = []
    last_fii_div_list: list = []
    value_div_fii_list: list = []
    magic_number_fii_list: list = []
    price_magic_number_fii_list: list = []

    for fii in fiis:
        # Ultima valor de fechamento da acao
        stonk_value: float = round(stonk_last_value(fii, '01-01-2015').iloc[-1].Close, 2)
        last_fii_value_list.append(stonk_value)

        df = dividendos_fii(fii).query('date > 2015')

        # Ultimo dividendo
        last_div = dividendos_fii(fii).query('date > 2015').valor.iloc[0]
        last_fii_div_list.append(last_div)

        # Mediana dos dividendos por mes
        median_by_year = df.groupby(df.date.map(lambda x: x.year)).median()
        median_by_month_value: float = float(median_by_year.median())
        median_fii_list.append(median_by_month_value)

        # value_div: las_value / last_div
        value_div: float = (last_div / stonk_value) * 100
        value_div_fii_list.append(value_div)

        # magic_number: valor_acao / dividendo_mes
        magic_number_value = int(stonk_value / median_by_month_value)
        magic_number_fii_list.append(magic_number_value)

        # Valor para o MG
        price_magic_number: float = round(stonk_value * magic_number_value, 2)
        price_magic_number_fii_list.append(price_magic_number)

        # Plots
        if plot:
            plot_by_dividendo(fii, 'fii')
            plot_by_year(fii, 'fii')

    # Criando o df
    df_fiis = pd.DataFrame([fii.upper() for fii in fiis], columns=['code'])
    df_fiis['last_value'] = pd.DataFrame(last_fii_value_list)
    df_fiis['last_div'] = pd.DataFrame(last_fii_div_list)
    df_fiis['value_div'] = pd.DataFrame(value_div_fii_list)
    df_fiis['median_div'] = pd.to_numeric(median_fii_list)
    df_fiis['magic_number'] = pd.to_numeric(magic_number_fii_list)
    df_fiis['price_magic_number'] = pd.to_numeric(price_magic_number_fii_list)

    return df_fiis

In [9]:
my_fii()

,code,last_value,last_div,value_div,median_div,magic_number,price_magic_number
0,ALZR11,117.08,0.67,0.572258,0.6050,193,22596.44
1,BCFF11,76.55,0.50,0.653168,0.5100,150,11482.50
2,HGLG11,165.50,1.00,0.604230,0.7800,212,35086.00
3,XPLG11,105.20,0.61,0.579848,0.5950,176,18515.20
4,XPIN11,98.49,0.62,0.629506,0.6200,158,15561.42
5,MXRF11,9.91,0.07,0.706357,0.0700,141,1397.31
6,HFOF11,92.75,0.60,0.646900,0.6325,146,13541.50
7,RECT11,74.99,0.60,0.800107,0.8100,92,6899.08
8,HCTR11,130.00,1.90,1.461538,1.4950,86,11180.00
9,KNIP11,106.90,1.13,1.057063,0.7475,143,15286.70


In [10]:
my_fii().sort_values(by=['value_div'], ascending=False)

,code,last_value,last_div,value_div,median_div,magic_number,price_magic_number
8,HCTR11,130.00,1.90,1.461538,1.4950,86,11180.00
9,KNIP11,106.90,1.13,1.057063,0.7475,143,15286.70
10,IRDM11,119.01,1.15,0.966305,0.8700,136,16185.36
7,RECT11,74.99,0.60,0.800107,0.8100,92,6899.08
11,VINO11,57.64,0.45,0.780708,0.3600,160,9222.40
5,MXRF11,9.91,0.07,0.706357,0.0700,141,1397.31
1,BCFF11,76.55,0.50,0.653168,0.5100,150,11482.50
6,HFOF11,92.75,0.60,0.646900,0.6325,146,13541.50
4,XPIN11,98.49,0.62,0.629506,0.6200,158,15561.42
2,HGLG11,165.50,1.00,0.604230,0.7800,212,35086.00


In [11]:
df_sorted = dividendos_acao('bbse3').sort_values(by='date')
df_value = stonk_last_value('bbse3')

In [12]:
df_value

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2013-04-29,16.950001,16.100000,16.950001,16.600000,85582800.0,9.656610
2013-04-30,17.080000,16.600000,16.600000,17.049999,26950200.0,9.918386
2013-05-02,17.500000,16.980000,17.000000,17.500000,16057200.0,10.180161
2013-05-03,17.850000,17.370001,17.420000,17.750000,13031000.0,10.325591
2013-05-06,17.990000,17.639999,17.750000,17.930000,19911900.0,10.430304
...,...,...,...,...,...,...
2021-06-22,25.570000,25.200001,25.410000,25.540001,5243800.0,25.540001
2021-06-23,25.500000,24.660000,25.410000,24.700001,7326800.0,24.700001
2021-06-24,24.860001,24.090000,24.730000,24.400000,5502100.0,24.400000


In [13]:
data = [go.Scatter(
            x=df_sorted.date,
            y=df_sorted.valor
            )
        ]

layout = dict(
    autosize=False,
    width=1500,
    height=900,
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [14]:
div = go.Scatter(
    x=df_sorted.date,
    y=df_sorted.valor,
    name = "TAEE4 - div",
    line = dict(color = '#330000'),
    opacity = 0.8
)

value = go.Scatter(
    x=df_value.index,
    y=df_value.Close,
    name = "TAEE4 - Value",
    line = dict(color = '#17BECF'),
    opacity = 0.8
)

data = [div, value]

layout = dict(
    autosize=False,
    width=1500,
    height=900,
    title="TAESA",
    title_x=0.5,
    xaxis = dict(
        range = ['2020-01-01','2020-12-31']
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)

In [20]:
div = go.Scatter(
    x=df_sorted.date,
    y=df_sorted.valor,
    name = "TAEE4 - div",
    line = dict(color = '#330000'),
    opacity = 0.8
)

# value = go.Scatter(
#     x=df_value.index,
#     y=df_value.Close,
#     name = "TAEE4 - Value",
#     line = dict(color = '#17BECF'),
#     opacity = 0.8
# )

data = [div]

layout = dict(
    autosize=False,
    width=1500,
    height=900,
    title="TAESA",
    title_x=0.5,
    xaxis = dict(
        range = ['2020-01-01','2021-06-31']
    )
)

fig = dict(data=data, layout=layout)
py.iplot(fig)